## 50.データの入手・整形

In [80]:
import pandas as pd
from sklearn.model_selection import train_test_split

FIRST_TIME = False #ファイルの保存などの処理の実行有無
DATA_PATH = "./data/news_aggregator_dataset/"

df_src = pd.read_csv(DATA_PATH + "newsCorpora_re.csv", header=None, sep="\t", names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])

In [81]:
df = df_src[df_src["PUBLISHER"].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]

seed = 144
train_df, test_valid = train_test_split(df, test_size=0.2, shuffle=True, random_state=seed, stratify=df["CATEGORY"])
test_df, valid_df = train_test_split(test_valid, test_size=0.5, shuffle=True, random_state=seed, stratify=test_valid["CATEGORY"])

train_df.head()
print(f"train size:{train_df.shape}, valid size:{valid_df.shape}, test size:{test_df.shape}")

train size:(10684, 8), valid size:(1336, 8), test size:(1336, 8)


In [82]:
train_df = train_df.loc[:,["CATEGORY", "TITLE"]]
valid_df = valid_df.loc[:,["CATEGORY", "TITLE"]]
test_df = test_df.loc[:,["CATEGORY", "TITLE"]]

if FIRST_TIME:
    train_df.to_csv(DATA_PATH + "train.txt", sep="\t", index=False)
    valid_df.to_csv(DATA_PATH + "valid.txt", sep="\t", index=False)
    test_df.to_csv(DATA_PATH + "test.txt", sep="\t", index=False)

In [83]:
print("train:")
print(train_df['CATEGORY'].value_counts())
print("\nvalid:")
print(valid_df['CATEGORY'].value_counts())
print("\ntest")
print(test_df['CATEGORY'].value_counts())

train:
b    4501
e    4235
t    1220
m     728
Name: CATEGORY, dtype: int64

valid:
b    563
e    529
t    153
m     91
Name: CATEGORY, dtype: int64

test
b    563
e    530
t    152
m     91
Name: CATEGORY, dtype: int64


In [84]:
print(train_df.head(5))

       CATEGORY                                              TITLE
149350        b  Airline Mergers Pushed JetBlue's Pilots to Joi...
175696        e      Andrew Garfield wants Black-HispanicSpiderman
371351        b  Korean Bonds Fall as Choi Refrains From Signal...
275697        m  REFILE-AstraZeneca fights to regain cancer mar...
274356        e  Beyonce and Solange both take the plunge at fi...


## 51.特徴量抽出
TFIDFを計算し、特徴量とする

In [85]:
import string
import re

def preprocessing(text):
    text = re.sub(r"[-/:-@[-`'{-~]", r' ', text) #記号を空白に
    text = re.sub(r' {2,}', r' ', text) #2連続以上の空白を1つの空白に
    text = text.lower()
    text = re.sub(r'[0-9]+', '0', text)
    text = re.sub(r'^\s+', "", text)
    return text

print(preprocessing("Rolf Harris led 'a war on child molesters': Hypocrisy of star honoured by Queen  ..."))

rolf harris led a war on child molesters hypocrisy of star honoured by queen ...


In [86]:
train_df["TITLE"] = train_df["TITLE"].map(lambda x: preprocessing(x))
valid_df["TITLE"] = valid_df["TITLE"].map(lambda x: preprocessing(x))
test_df["TITLE"] = test_df["TITLE"].map(lambda x: preprocessing(x))

print(train_df.head(5))

       CATEGORY                                              TITLE
149350        b  airline mergers pushed jetblue s pilots to joi...
175696        e      andrew garfield wants black hispanicspiderman
371351        b  korean bonds fall as choi refrains from signal...
275697        m  refile astrazeneca fights to regain cancer mar...
274356        e  beyonce and solange both take the plunge at fi...


In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer

cut_off = 5
tfidf = TfidfVectorizer(min_df=cut_off, ngram_range=(1, 2))

train_vec = tfidf.fit_transform(train_df["TITLE"])
valid_vec = tfidf.transform(valid_df["TITLE"])
test_vec = tfidf.transform(test_df["TITLE"])

train_vec_df = pd.DataFrame(train_vec.toarray(), columns=tfidf.get_feature_names())
valid_vec_df = pd.DataFrame(valid_vec.toarray(), columns=tfidf.get_feature_names())
test_vec_df = pd.DataFrame(test_vec.toarray(), columns=tfidf.get_feature_names())

print(train_vec_df.head())

   0bn   0d   0k   0m  0million  0nd  0nd day  0rd   0s  0st  ...  zac efron  \
0  0.0  0.0  0.0  0.0       0.0  0.0      0.0  0.0  0.0  0.0  ...        0.0   
1  0.0  0.0  0.0  0.0       0.0  0.0      0.0  0.0  0.0  0.0  ...        0.0   
2  0.0  0.0  0.0  0.0       0.0  0.0      0.0  0.0  0.0  0.0  ...        0.0   
3  0.0  0.0  0.0  0.0       0.0  0.0      0.0  0.0  0.0  0.0  ...        0.0   
4  0.0  0.0  0.0  0.0       0.0  0.0      0.0  0.0  0.0  0.0  ...        0.0   

   zealand  zebra  zendaya  zeppelin  zero  zone  zone bond  zone bonds  \
0      0.0    0.0      0.0       0.0   0.0   0.0        0.0         0.0   
1      0.0    0.0      0.0       0.0   0.0   0.0        0.0         0.0   
2      0.0    0.0      0.0       0.0   0.0   0.0        0.0         0.0   
3      0.0    0.0      0.0       0.0   0.0   0.0        0.0         0.0   
4      0.0    0.0      0.0       0.0   0.0   0.0        0.0         0.0   

   zone inflation  
0             0.0  
1             0.0  
2       

In [88]:
if FIRST_TIME:
    train_vec_df.to_csv(DATA_PATH + "train.feature.txt", sep="\t", index=False)
    valid_vec_df.to_csv(DATA_PATH + "valid.feature.txt", sep="\t", index=False)
    test_vec_df.to_csv(DATA_PATH + "test.feature.txt", sep="\t", index=False)
# train.feature.txtは100MBを越えるのでgit管理しない 必要があればローカルで生成する

## 52.学習
ロジスティック回帰モデルを学習する

In [89]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=seed, max_iter=10000)
model.fit(train_vec_df, train_df["CATEGORY"])

LogisticRegression(max_iter=10000, random_state=144)

## 53.予測
学習したモデルを使って予測を行う

In [90]:
train_pred = model.predict(train_vec_df)
test_pred = model.predict(test_vec_df)

## 54.正解率の計算
学習データと評価データについて正解率を学習する

In [91]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(train_df["CATEGORY"], train_pred)
test_accuracy = accuracy_score(test_df["CATEGORY"], test_pred)

print(f"train accuracy: {train_accuracy}")
print(f"test accuracy: {test_accuracy}")


train accuracy: 0.940752527143392
test accuracy: 0.8802395209580839
